<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_04/TS%20-%20W4%20-%2010%20-%20Manchas%20Solares%20com%20DNN%20e%20LSTM%20(Laborat%C3%B3rio%203).ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Previsão de manchas solares com redes neurais

A esta altura do curso, você deve ser capaz de explorar diferentes arquiteturas de rede para previsão.

Nos laboratórios anteriores, você usou DNNs, RNNs e CNNs para criar esses modelos diferentes.

No laboratório de prática final deste módulo de séries temporais, você tentará mais uma configuração, que é uma combinação de todos esses tipos de redes:
* as janelas de dados passarão por uma convolução,
* seguida por LSTMs empilhados, seguidos por camadas densas empilhadas.

Veja se isso melhora os resultados ou se você pode simplesmente optar por modelos mais simples.

## Importações

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv

## Utilitários

In [ ]:
def plot_series(x, y, format="-", start=0, end=None,
                title=None, xlabel=None, ylabel=None, legend=None ):
    """
    Visualiza dados de séries temporais

    Args:
      x (array of int) - contém valores para o eixo x
      y (matriz de int ou tupla de matrizes) - contém os valores para o eixo y
      format (string) - estilo de linha ao plotar o gráfico
      start (int) - primeiro passo de tempo a ser plotado
      end (int) - último passo de tempo a ser plotado
      title (string) - título do gráfico
      xlabel (string) - rótulo do eixo x
      ylabel (string) - rótulo para o eixo y
      legend (lista de strings) - legenda para o gráfico
    """

    # Configurar as dimensões da figura do gráfico
    plt.figure(figsize=(10, 6))

    # Verificar se há mais de duas séries para plotar
    if type(y) is tuple:

      # Fazer um loop sobre os elementos y
      for y_curr in y:

        # Plote os valores x e y atuais
        plt.plot(x[start:end], y_curr[start:end], format)

    else:
      # Plote os valores x e y
      plt.plot(x[start:end], y[start:end], format)

    # Rotular o eixo x
    plt.xlabel(xlabel)
    
    # Rotular o eixo y
    plt.ylabel(ylabel)

    # Definir a legenda
    if legend:
      plt.legend(legend)

    # Definir o título
    plt.title(title)

    # Sobrepor uma grade no gráfico
    plt.grid(True)

    # Desenhe o gráfico na tela
    plt.show()

## Faça o download e visualize o conjunto de dados

In [ ]:
# Faça o download do conjunto de dados
!wget https://storage.googleapis.com/tensorflow-1-public/course4/Sunspots.csv

In [ ]:
# Initialize lists
time_step = []
sunspots = []

# Abrir arquivo CSV
with open('./Sunspots.csv') as csvfile:

  # Inicializar o leitor
  reader = csv.reader(csvfile, delimiter=',')

  # Pular a primeira linha
  next(reader)

  # Acrescente o número da linha e de sunspots às listas
  for row in reader:
    time_step.append(int(row[0]))
    sunspots.append(float(row[2]))

# Converter listas em matrizes numéricas
time = np.array(time_step)
series = np.array(sunspots)

# Visualizar os dados
plot_series(time, series, xlabel='Month', ylabel='Média mensal do número total de manchas solares')

## Split the Dataset

In [ ]:
# Definir o tempo de divisão
split_time = 3000

# Obter o conjunto de treino
time_train = time[:split_time]
x_train = series[:split_time]

# Obter o conjunto de validação
time_valid = time[split_time:]
x_valid = series[split_time:]

## Prepare Features and Labels

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Gera janelas de conjunto de dados

    Args:
      series (matriz de float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo a serem incluídas no recurso
      batch_size (int) - o tamanho do lote
      shuffle_buffer(int) - tamanho do buffer a ser usado para o método shuffle

    Retorna:
      dataset (TF Dataset) - Conjunto de dados TF contendo janelas de tempo
    """

    # Gerar um conjunto de dados TF a partir dos valores da série
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Janela de dados, mas só pega aqueles com o tamanho especificado
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    # Achatar as janelas, colocando seus elementos em um único lote
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Criar tuplas com recursos e rótulos
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Embaralhar as janelas
    dataset = dataset.shuffle(shuffle_buffer)

    # Criar lotes de janelas
    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

Conforme mencionado nas aulas, se os resultados não forem bons, você pode tentar ajustar os parâmetros aqui e ver se o modelo aprenderá melhor.

In [ ]:
# Hipermarâmetros
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

# Gerar as janelas do conjunto de dados
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

## Construir o modelo

Você já viu essas camadas antes e aqui está a aparência delas quando combinadas.

In [ ]:
# Construir o modelo
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1,
                      activation="relu",
                      padding='causal',
                      input_shape=[window_size, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])

 # Imprimir o resumo do modelo
model.summary()

## Ajuste a taxa de aprendizado

Como de costume, você deverá escolher uma taxa de aprendizado ideal.

In [ ]:
# Obter pesos iniciais
init_weights = model.get_weights()

In [ ]:
# Definir o agendador de taxa de aprendizado
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Inicializar o otimizador
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Treinar o modelo
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
# Definir a matriz de taxa de aprendizado
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Definir o tamanho da figura
plt.figure(figsize=(10, 6))

# Definir a grade
plt.grid(True)

# Plotar a perda em escala logarítmica
plt.semilogx(lrs, history.history["loss"])

# Aumentar o tamanho dos tickmarks
plt.tick_params('both', length=10, width=1, which='both')

# Definir os limites do gráfico
plt.axis([1e-8, 1e-3, 0, 100])

## Treinar o modelo

Agora você pode continuar a redefinir e treinar o modelo. Ele está definido para 500 épocas na célula abaixo, mas fique à vontade para aumentá-lo, se desejar.

In [ ]:
# Redefinir estados gerados pelo Keras
tf.keras.backend.clear_session()

# Redefinir os pesos
model.set_weights(init_weights)

In [ ]:
# Definir a taxa de aprendizado
learning_rate = 8e-7

# Definir o otimizador
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
# Treinar o modelo
history = model.fit(train_set,epochs=500)

Você pode visualizar o treinamento e ver se a perda e o MAE ainda estão em tendência de queda.

In [ ]:
# Obtenha a receita e a perda do registro histórico
mae=history.history['mae']
loss=history.history['loss']

# Obter o número de épocas
epochs=range(len(loss))

# Plotar mae e loss
plot_series(
    x=epochs,
    y=(mae, loss),
    title='MAE e Loss',
    xlabel='MAE',
    ylabel='Loss',
    legend=['MAE', 'Loss']
    )

# Plote apenas os últimos 80% das épocas
zoom_split = int(epochs[-1] * 0.2)
epochs_zoom = epochs[zoom_split:]
mae_zoom = mae[zoom_split:]
loss_zoom = loss[zoom_split:]

# Plotar a receita e a perda com zoom
plot_series(
    x=epochs_zoom,
    y=(mae_zoom, loss_zoom),
    title='MAE and Loss',
    xlabel='MAE',
    ylabel='Loss',
    legend=['MAE', 'Loss']
    )

## Previsão do modelo

Como antes, você pode obter as previsões para o intervalo de tempo do conjunto de validação e calcular as métricas.

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    """Usa um modelo de entrada para gerar previsões em janelas de dados

    Args:
      model (TF Keras Model) - modelo que aceita janelas de dados
      series (array of float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo a serem incluídas na janela
      batch_size (int) - o tamanho do lote

    Retorna:
      forecast (matriz numpy) - matriz que contém as previsões
    """

    # Gerar um conjunto de dados TF a partir dos valores da série
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Janela de dados, mas só pega aqueles com o tamanho especificado
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)

    # Achatar as janelas, colocando seus elementos em um único bloco
    dataset = dataset.flat_map(lambda w: w.batch(window_size))
    
    # Criar lotes de janelas
    dataset = dataset.batch(batch_size).prefetch(1)

    # Obter previsões em todo o conjunto de dados
    forecast = model.predict(dataset)

    return forecast

In [ ]:
# Reduzir a série original
forecast_series = series[split_time-window_size:-1]

# Use a função auxiliar para gerar previsões
forecast = model_forecast(model, forecast_series, window_size, batch_size)

# Soltar eixo unidimensional
results = forecast.squeeze()

# Plotar os resultados
plot_series(time_valid, (x_valid, results))

In [ ]:
# Calcular o MAE
print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())

## Taxa de aprendizagem dinâmica

Nesta seção, você verá outra maneira de definir dinamicamente a taxa de aprendizagem.

Como você deve ter notado, o treinamento por um longo período gera cada vez menos alterações na perda e nas métricas. 

Você pode executar a célula abaixo para observar isso novamente.

In [ ]:
# Inicializar o otimizador
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-7, momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# Treinar o modelo
history = model.fit(train_set,epochs=10)

Em algum momento, a taxa de aprendizagem estática que você definiu pode não ser mais a ideal quando o modelo estiver aprendendo há algum tempo.

Talvez você queira diminuí-la um pouco mais para ver melhorias melhores.

Uma maneira de realizar isso é fazer com que seu loop de treinamento diminua gradualmente a taxa de aprendizagem por época.

Você pode passar uma função lambda semelhante à que você fez para o agendador da taxa de aprendizado anteriormente ou usar [ExponentialDecay()](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay). Esse é um agendador integrado da API do Keras. Ele decairá a taxa de aprendizado definida por essa função:

```
def decayed_learning_rate(step):
  return initial_learning_rate * decay_rate ^ (step / decay_steps)
```

Veja como ela é usada abaixo.

In [ ]:
# Definir a taxa de aprendizado inicial
initial_learning_rate=1e-7

# Definir o agendador
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=400,
    decay_rate=0.96,
    staircase=True)

# Definir o otimizador
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# Treinar o modelo
history = model.fit(train_set,epochs=40)

Agora veja os resultados obtendo previsões e calculando as métricas.

In [ ]:
# Reduzir a série original
forecast_series = series[split_time-window_size:-1]

# Use a função auxiliar para gerar previsões
forecast = model_forecast(model, forecast_series, window_size, batch_size)

# Descarte eixo unidimensional
results = forecast.squeeze()

# Plote os resultados
plot_series(time_valid, (x_valid, results))

In [ ]:
# Calcular o MAE
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()

## Encerramento

Você implementou uma arquitetura profunda e complexa composta por CNNs, RNNs e DNNs.